<a href="https://colab.research.google.com/github/sibyjackgrove/SolarPV-DER-simulation-utility/blob/master/examples/PV-DER_usage_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Accessing and updating model parameters

### Import classes
Import classes neccessary to define instances of the PV-DER model, events, simulation, and results.

In [1]:
from pvder.DER_wrapper import DERModel

from pvder.grid_components import Grid
from pvder.dynamic_simulation import DynamicSimulation
from pvder.simulation_events import SimulationEvents
from pvder.simulation_utilities import SimulationUtilities,SimulationResults
from pvder import utility_functions
import pvder

#Import Python modules
import logging
import numpy as np
import scipy as sci
import math
import cmath
import pandas


import matplotlib as matplotlib
print('Scipy version:',sci.__version__)
print('Numpy version:',np.__version__)
print('Matplotlib version:',matplotlib.__version__)
print('PVDER version:',pvder.__version__)
#%matplotlib inline    #To show interactive plot in the notebook cell

Scipy version: 1.4.1
Numpy version: 1.20.3
Matplotlib version: 3.4.2
PVDER version: 0.5.5


### Specify voltage values
Specify voltage values for non-standalone model.

In [2]:
logging.debug('test')
Va=164.78-124.57j
Vb=-190.96-78.26j
Vc=26.24+206.56j
Vrms = abs(Va)/math.sqrt(2)
print('Vrms:{:.2f}'.format(Vrms))
print('Va:{:.2f},Vb:{:.2f},Vc:{:.2f}'.format(Va,Vb,Vc))
print('V0:{:.2f}'.format(Va+Vb+Vc))

events1 = SimulationEvents(verbosity = 'DEBUG')
grid1 = Grid(events=events1,unbalance_ratio_b=1.0,unbalance_ratio_c=1.0)


Vrms:146.07
Va:164.78-124.57j,Vb:-190.96-78.26j,Vc:26.24+206.56j
V0:0.06+3.73j


## Select options

In [3]:
STAND_ALONE = False
STEADY_STATE = True
LOOP=False

model_type = 'ThreePhaseUnbalanced' #Model type
#three phase model types: 'ThreePhaseBalanced','ThreePhaseUnbalanced','ThreePhaseUnbalancedConstantVdc'
#single phase model types: 'SinglePhase'

der_verbosity = 'INFO' #Logging verbosity
config_file = r'../config_der.json'
parameter1 = '50'#'50_type1'#'50','10' '50_constantVdc','50_balanced','250'

### Create objects required to simulate either single phase or three phase PV-DER

In [4]:
PV_DER1 = DERModel(modelType=model_type,events=events1,configFile=config_file,
                       Vrmsrated = Vrms,
                       gridVoltagePhaseA = Va,gridVoltagePhaseB = Vb,gridVoltagePhaseC = Vc,gridFrequency=2*math.pi*60.0,
                       derId=parameter1,
                       standAlone = STAND_ALONE,steadyStateInitialization=STEADY_STATE,
                       verbosity = der_verbosity)        
    
sim1 = DynamicSimulation(gridModel=grid1,PV_model=PV_DER1.DER_model,events = events1,verbosity = 'INFO',solverType='odeint',LOOP_MODE=LOOP) #'odeint','ode-vode-bdf'
results1 = SimulationResults(simulation = sim1,PER_UNIT=True,verbosity = 'INFO')

INFO:pvder_logger:DER configuration with ID:50 was found in ../config_der.json
INFO:pvder_logger:-SolarPVDERThreePhase_1:Instance created with DER parameter ID: 50; Specifications - Srated:50.0 kVA, Ppv:45.8 kW, Vrms:146.1 V, Steady state:True,LVRT Enable:True,HVRT Enable:True


Optimization results:[ 0.6030582  -0.45266374  1.16945499 -0.89241647]


## Create and use a user defined parameter dictionary
### 1. Show existing parameter dictionaries using show_parameter_dictionaries() and show_parameter_types().


In [5]:
PV_DER1.DER_model.show_parameter_dictionaries()
PV_DER1.DER_model.show_parameter_types()

-----Parameter dictionary: Parameter IDs-----
module_parameters : 50
inverter_ratings : 50
circuit_parameters : 50
controller_gains : 50
steadystate_values : 50
-----Parameter dictionary: Parameter types-----
module_parameters : Np,Ns,Vdcmpp0,Vdcmpp_min,Vdcmpp_max
inverter_ratings : Srated,Vdcrated,Ioverload,Vrmsrated,Iramp_max_gradient_real,Iramp_max_gradient_imag
circuit_parameters : Rf_actual,Lf_actual,C_actual,R1_actual,X1_actual,Z1_actual
controller_gains : Kp_GCC,Ki_GCC,Kp_DC,Ki_DC,Kp_Q,Ki_Q,wp
steadystate_values : maR0,maI0,iaR0,iaI0


### 2. Get parameter values using get_parameter_values()

In [6]:
DER_parameters = PV_DER1.DER_model.get_parameter_dictionary(parameter_type='all',parameter_ID='50')

{   'C_actual': 0.0003,
    'Ioverload': 1.3,
    'Iramp_max_gradient_imag': 1.0,
    'Iramp_max_gradient_real': 1.0,
    'Ki_DC': -10.0,
    'Ki_GCC': 2000.0,
    'Ki_Q': 10.0,
    'Kp_DC': -2.0,
    'Kp_GCC': 6000.0,
    'Kp_Q': 0.2,
    'Lf_actual': 2.5e-05,
    'Np': 11,
    'Ns': 735,
    'R1_actual': 0.0019,
    'Rf_actual': 0.002,
    'Srated': 50000.0,
    'Vdcmpp0': 550.0,
    'Vdcmpp_max': 650.0,
    'Vdcmpp_min': 525.0,
    'Vdcrated': 550.0,
    'Vrmsrated': 146.06528215150922,
    'X1_actual': 0.0561,
    'Z1_actual': (0.0019+0.0561j),
    'iaI0': 0.001,
    'iaR0': 1.0,
    'maI0': 0.0,
    'maR0': 0.89,
    'wp': 200000.0}


### 3. Select name of new parameter dictionary (e.g. 'my_DER') and source parameter ID (e.g. '50').
### 4. Create and initialize new parameter dictionary using initialize_parameter_dict().

In [7]:
PV_DER1.DER_model.initialize_parameter_dict(parameter_ID='my_DER',source_parameter_ID='50')

INFO:pvder_logger:-SolarPVDERThreePhase_1:Created and initialized new parameter dicitonary my_DER with source dictionary 50.


### 5. Update the new parameter dictionary with desired values (e.g. {'Np':5} in 'module_parameters' and {'C_actual':5} in 'circuit_parameters') using update_parameter_dict().

In [8]:
PV_DER1.DER_model.update_parameter_dict(parameter_ID='my_DER',parameter_type='module_parameters',parameter_dict={'Np':5})
PV_DER1.DER_model.update_parameter_dict(parameter_ID='my_DER',parameter_type='circuit_parameters',parameter_dict={'C_actual':0.0005})

### 6. Update the PV-DER model with parameters from the new parameter dictionary using modify_DER_parameters()

In [9]:
PV_DER1.DER_model.modify_DER_parameters(parameter_ID='my_DER')

INFO:pvder_logger:-SolarPVDERThreePhase_1:PV-DER parameters updated with parameters fromparameter dictionary my_DER!


Optimization results:[ 0.60092891 -0.45449525  0.5319794  -0.40595435]


### 7. Save and load parameter dictionaries using  save_parameter_dictionary() and load_parameter_dictionary()

In [10]:
file_name = PV_DER1.DER_model.save_parameter_dictionary(parameter_ID='my_DER',save_format='pickle')

INFO:pvder_logger:-SolarPVDERThreePhase_1:Saved all the parameter dicitonaries as a pickle file in my_DER.pkl.


In [11]:
_=PV_DER1.DER_model.load_parameter_dictionary(file_name='my_DER.pkl')

INFO:pvder_logger:-SolarPVDERThreePhase_1:Created and initialized new parameter dicitonary my_DER with source dictionary 50.
INFO:pvder_logger:-SolarPVDERThreePhase_1:Succesfully loaded parameters from my_DER.pkl into DER parameter dictionary with parameter ID my_DER.


Read following dictionary from my_DER.pkl:
{   'C_actual': 0.0005,
    'Ioverload': 1.3,
    'Iramp_max_gradient_imag': 1.0,
    'Iramp_max_gradient_real': 1.0,
    'Ki_DC': -10.0,
    'Ki_GCC': 2000.0,
    'Ki_Q': 10.0,
    'Kp_DC': -2.0,
    'Kp_GCC': 6000.0,
    'Kp_Q': 0.2,
    'Lf_actual': 2.5e-05,
    'Np': 5,
    'Ns': 735,
    'R1_actual': 0.0019,
    'Rf_actual': 0.002,
    'Srated': 50000.0,
    'Vdcmpp0': 550.0,
    'Vdcmpp_max': 650.0,
    'Vdcmpp_min': 525.0,
    'Vdcrated': 550.0,
    'Vrmsrated': 146.06528215150922,
    'X1_actual': 0.0561,
    'Z1_actual': (0.0019+0.0561j),
    'iaI0': 0.001,
    'iaR0': 1.0,
    'maI0': 0.0,
    'maR0': 0.89,
    'wp': 200000.0}


## Specify run time flags, simulation time, and run simulation 

In [12]:
%%time
sim1.jacFlag = True
sim1.DEBUG_SIMULATION = False
sim1.PER_UNIT = True
sim1.DEBUG_SOLVER  = False
sim1.tStop = 10.0
sim1.tInc = 1/120.# 0.001
sim1.run_simulation()
PV_DER1.DER_model.validate_model()
PV_DER1.DER_model.show_PV_DER_states('voltage')
PV_DER1.DER_model.show_PV_DER_states('current')
PV_DER1.DER_model.show_PV_DER_states('power')
PV_DER1.DER_model.show_PV_DER_states('duty cycle')

None-sim_1:Simulation started at 0.0 s and will end at 10.0 s
None-sim_1:Simulation was completed in 00:00:00
Active power output error:0.2776
Reactive power output error:0.0002
Inverter filter active power loss error:0.0000
Inverter filter reactive power loss error:0.0000

______-SolarPVDERThreePhase_1 - Voltage_____
Vdc:550.00
Vta:165.27-124.15j V
Vtb:-190.84-78.89j V,Vtb:25.63+206.77j V
Vtn:0.06+3.73j V
Va:164.78-124.57j V
Vb:-190.96-78.26j V,Vc:26.24+206.56j V
Vn:0.06+3.73j V
Vtrms:146.51 V
Vpccrms:146.41 V

______-SolarPVDERThreePhase_1 - Current_____
ia:53.20-40.60j A
ib:-61.76-25.77j A,ic:8.56+66.37j A
In:-0.00-0.00j A
Irms:47.32 V

______-SolarPVDERThreePhase_1 - Power_____
Ppv:20797.7 W
S:20797.7+63.3j VA
S_PCC:20784.2-0.0j VA

______-SolarPVDERThreePhase_1 - Duty cycle_____
ma:0.60-0.45j
mb:-0.69-0.29j,mc:0.09+0.75j
m0:0.00+0.01j
CPU times: user 1.27 s, sys: 5.61 s, total: 6.88 s
Wall time: 450 ms


/home/splathottam/.local/lib/python3.7/site-packages/scipy/integrate/odepack.py:251: ODEintWarning: Integration successful.
  warnings.warn(warning_msg, ODEintWarning)


### Get trajectories for current, voltage, and power as a dictionary

In [13]:
trajectories = sim1.get_trajectories()